# Cloudless images from Sentinel HUB

Find cloudless images for a selected AOI in a selected time frame.

The conda-forge version of SentinelHub package is not the latest version (problem with access token for Landsat service).

Install the latest version (3.4) using pip:

`$ pip install sentinelhub --upgrade` or `$ pip install sentinelhub --upgrade --user`

>**WARNING**: If setting the credentials manually be careful NOT TO PUBLICALLY SHARE the repository or this script!

In [1]:
from sentinelhub import SHConfig
import glob

config = SHConfig()

# Read from file if it exist, otherwise set manually
user_creds = glob.glob(".\\userdata\\credentials*.txt")
if user_creds:
    with open(user_creds[0], "r") as txt:
        instance_id, shub_id, shub_secret = txt.readline().split(" ")
        config.instance_id = instance_id
        config.sh_client_id = shub_id
        config.sh_client_secret = shub_secret
else:
    config.instance_id = '<your instance id>'
    config.sh_client_id = '<your client id>'
    config.sh_client_secret = '<your client secret>'

config.sh_base_url = 'https://services-uswest2.sentinel-hub.com'

print("Instance ID: " + config.instance_id)

Instance ID: CEOIS2_b


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# import os
# import datetime
# import numpy as np
# import matplotlib.pyplot as plt
# from utils import plot_image
# import osmnx as ox
# import glob
# import rasterio
# import rasterio.plot

from sentinelhub import CRS, BBox, bbox_to_dimensions, DataCollection, \
    SentinelHubCatalog, filter_times, \
    SentinelHubRequest, MimeType, SentinelHubDownloadClient

import datetime as dt
#import tarfile
from os.path import join, dirname
from os import rename
from shutil import rmtree

import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from sentinelhub import SentinelHubCatalog, BBox, Geometry, SHConfig, CRS, DataCollection
from sentinelhub.time_utils import serialize_time, parse_time_interval, parse_time
from sentinelhub.sh_utils import FeatureIterator, remove_undefined
from sentinelhub.sentinelhub_catalog import CatalogSearchIterator

import json
from dateutil.parser import parse

## Prepare input data

In [4]:
# Data collection
data_collection = DataCollection.LANDSAT_OT_L2  # DataCollection.SENTINEL2_L2A

# Resolution (pixel size)
resolution = 10

# # CRS for sentinelhub query getattr() assigns module CRS with UTM_### method ==> CRS.UTM_36N
# utm_zone = "36N"
# sample_crs = getattr(CRS, f"UTM_{utm_zone}")
# # List of supported CRSs: https://docs.sentinel-hub.com/api/latest/api/process/crs/

# ------------------------------------------------------------------------------------------------

# ## Build bbox from centrepoint + distance
# # centrepoint
# utm_x = 512817
# utm_y = 269983
# # AOI size in meters (x, y)
# aoi_size = (5120, 5120)

# # Round UTM coords to nearest 10 (so the raster will look nice in UTM - "target aligned pixels")
# x_y = (round(utm_x, -1), round(utm_y, -1)) 

# bbox_utm = [x_y[0] - aoi_size[0]/2,
#             x_y[1] - aoi_size[1]/2,
#             x_y[0] + aoi_size[0]/2,
#             x_y[1] + aoi_size[1]/2]


# ## Use this to create a bbox directly:
# bbox_utm = [510260, 267420, 515380, 272540]

# ------------------------------------------------------------------------------------------------

# Prepare some data for sentinelhub
# sample_bbox = BBox(bbox=bbox_utm, crs=sample_crs)
sample_bbox = BBox(bbox=[12.44693, 41.870072, 12.541001, 41.917096], crs=CRS.WGS84)
sample_size = bbox_to_dimensions(sample_bbox, resolution=resolution)
print(sample_size)
print(sample_bbox)
# print(sample_crs)

(796, 499)
12.44693,41.870072,12.541001,41.917096


## Search for all available images in a time frame

Kako pa dobimo vse posnetke (vse datume) znotraj enega četrtletja?

In [52]:
%%time
# Sarch catalog for all images with bbox in a time interval
catalog = SentinelHubCatalog(config=config)

# SET TIME INTERVAL
time = ('2019-01-01', '2019-12-31')
collection = data_collection

# # ----------------------------------------------------------------------------------------------

# # Set up credentials for use with batch
# client = BackendApplicationClient(client_id=config.sh_client_id)
# oauth = OAuth2Session(client=client)

# # Fetch a token
# token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
#                           client_id=config.sh_client_id, client_secret=config.sh_client_secret)

# # Set-up batch parameters: HERE we change the endpoint
# url = 'https://services-uswest2.sentinel-hub.com/api/v1/catalog/search'
# headers = {
#     "Authorization": "Bearer " + token['access_token'],
#     "Content-Type": "application/json"
# }

# collection_id = catalog._parse_collection_id(collection)
# start_time, end_time = serialize_time(parse_time_interval(time, allow_undefined=True), use_tz=True)
# if sample_bbox and sample_bbox.crs is not CRS.WGS84:
#     sample_bbox = sample_bbox.transform_bounds(CRS.WGS84)

# payload = {
#     'collections': [collection_id],
#     'datetime': f'{start_time}/{end_time}' if time else None,
#     'bbox': list(sample_bbox) if sample_bbox else None,
#     'fields': {"include": ["id", "properties.datetime"]},
#     'limit': 60,
# }
# #print(payload)

# response = oauth.request("POST", url, headers=headers, json=payload)

# #response.json()

# all_timestamps = [parse(feature["properties"]['datetime']) for feature in response.json()["features"]]

search_iterator = catalog.search(
    data_collection,
    bbox=sample_bbox,
    time=time,
    fields={
        "include": [
            "id",
            "properties.datetime",
            "properties.eo:cloud_cover"
        ],
        "exclude": []
    }

)

results = list(search_iterator)
print('Total number of results:', len(list(search_iterator)))
# print(results[:4])

# Many timestamps differ only for a few seconds. That is because they are from
# tiles in the same orbit acquisition. We want to join them together in a single timestamp.

time_difference = dt.timedelta(hours=1)
all_timestamps = search_iterator.get_timestamps()

unique_acquisitions = filter_times(all_timestamps, time_difference)
print(f"Number of unique acquisitions: {len(unique_acquisitions)}")

Total number of results: 44
Number of unique acquisitions: 44
Wall time: 1.16 s


In [58]:
[print(result["id"]) for result in results]

LC08_L2SP_190031_20191228_20200825_02_T1
LC08_L2SP_191031_20191219_20201023_02_T2
LC08_L2SP_191031_20191203_20200825_02_T1
LC08_L2SP_190031_20191126_20200825_02_T1
LC08_L2SP_191031_20191117_20200825_02_T2
LC08_L2SP_190031_20191110_20200825_02_T1
LC08_L2SP_191031_20191101_20200825_02_T1
LC08_L2SP_190031_20191025_20200825_02_T1
LC08_L2SP_191031_20191016_20200825_02_T1
LC08_L2SP_190031_20191009_20200825_02_T1
LC08_L2SP_191031_20190930_20200825_02_T1
LC08_L2SP_190031_20190923_20200826_02_T1
LC08_L2SP_191031_20190914_20200826_02_T1
LC08_L2SP_190031_20190907_20200826_02_T1
LC08_L2SP_191031_20190829_20200826_02_T1
LC08_L2SP_190031_20190822_20200827_02_T1
LC08_L2SP_191031_20190813_20200827_02_T1
LC08_L2SP_190031_20190806_20200827_02_T1
LC08_L2SP_191031_20190728_20200827_02_T1
LC08_L2SP_190031_20190721_20200827_02_T1
LC08_L2SP_191031_20190712_20200827_02_T1
LC08_L2SP_190031_20190705_20200827_02_T1
LC08_L2SP_191031_20190626_20200827_02_T1
LC08_L2SP_190031_20190619_20200827_02_T1
LC08_L2SP_191031

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Filter by cloud cover

### Download all CLMs

In [36]:
%%time

# Create a Process API request for each acquisition:
cm_evalscript = """
    //VERSION=3

    function setup() {
        return {
            input: ["BQA"],
            output: {
                bands: 1,
                sampleType: "UINT8"
            }
        };
    }

    function evaluatePixel(sample) {
        return [decodeLs8Qa(sample.BQA).cloud];
    }
"""

# Requests will be appenede to this list
process_requests = []

for timestamp in unique_acquisitions:
    request = SentinelHubRequest(
        evalscript=cm_evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection,
                time_interval=(timestamp - time_difference, timestamp + time_difference)
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF)
        ],
        bbox=sample_bbox,
        size=sample_size,
        config=config
    )
    process_requests.append(request)
    

# In order to efficiently download data for all requests in parallel, we extract download information and pass it to a download client.
client = SentinelHubDownloadClient(config=config)
download_requests = [request.download_list[0] for request in process_requests]
data = client.download(download_requests)

print(f"Downloaded: {len(data)}")

Downloaded: 44
Wall time: 5.8 s


We used the [decodeLs8Qa()](https://docs.sentinel-hub.com/api/latest/evalscript/functions/#decodels8qa) function to evaluate cloud cover of Landsat images.

> *cloud:*
>- 0  -  this condition doesn't exist
>- 1  -  this condition exists

In [49]:
print((data[0] == 1).sum())  # CLOUD
print((data[0] == 0).sum())  # NO CLOUD

8804
388400


### Filter by cloud cover

In [50]:
# Select threshold (e.g. 0.05 for less or equal than 5%)
threshold = 0
# Calculat cloud ratio for all images
cc_ratio = [(cc_image == 1).sum() / cc_image.size for cc_image in data]
# Select those below the threshold
lt5_cc = [(i, cc) for i, cc in zip(unique_acquisitions, cc_ratio) if cc <= threshold]

print(len(lt5_cc))
lt5_cc

28


[(datetime.datetime(2019, 1, 10, 9, 47, 3, 171000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 1, 26, 9, 46, 59, 247000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 2, 2, 9, 53, 8, 812000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 2, 11, 9, 46, 57, 109000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 2, 18, 9, 53, 6, 675000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 3, 6, 9, 53, 1, 985000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 3, 15, 9, 46, 47, 655000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 3, 22, 9, 52, 57, 519000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 3, 31, 9, 46, 44, 746000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 5, 9, 9, 52, 53, 219000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 5, 25, 9, 53, 2, 990000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 6, 3, 9, 46, 56, 674000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 6, 19, 9, 47, 2, 910000, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2019, 6, 26, 9,

## Download selected images (num)

Evalscript parameters: [https://docs.sentinel-hub.com/api/latest/evalscript/v3/#parameters](https://docs.sentinel-hub.com/api/latest/evalscript/v3/#parameters)

In [59]:
%%time

# Double curved brackets, so we can insert variables using .format() method where single brackets are present
my_evalscript = """
    //VERSION=3

    function setup() {{
        return {{
            input: [{{
                bands: ["B02","B03","B04"],
                units: "reflectance"  // "DN" for Digital Numbers, "reflectance" for normalised
            }}],
            output: {{
                id: "{0}",
                bands: 3,
                sampleType: "FLOAT32"  // "UINT16" for "DN", "FLOAT32" for "reflectance" 
            }}
        }};
    }}
    
    //NOT USED IF ONLY ONE TIFF IS DEFINED IN RESPONSES (uncomment in the SentinelHubRequest below)
    function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {{
        outputMetadata.userData = {{ "norm_factor":  inputMetadata.normalizationFactor }}
    }}

    function evaluatePixel(sample) {{
        return [sample.B04, sample.B03, sample.B02];
    }}
"""

data_folder = ".\\downloaded"

process_requests = []
save_locations = []
for timestamp, _ in lt5_cc:
    # Insert custom name for output file into evalscript
    my_string = timestamp.strftime("Landsat_%Y%m%dT%H%M%S_cloudless")
    evalscript = my_evalscript.format(my_string)
    
    # Define custom name for correct file naming
    my_folder = join(data_folder, my_string)
    
    request = SentinelHubRequest(
        data_folder=my_folder,
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection,
                time_interval=(timestamp - time_difference, timestamp + time_difference)
            )
        ],
        responses=[
            SentinelHubRequest.output_response(my_string, MimeType.TIFF),
            #SentinelHubRequest.output_response("userdata", MimeType.JSON)
        ],
        bbox=sample_bbox,
        size=sample_size,
        config=config
    )
    process_requests.append(request)
    save_locations.append(join(my_folder, request.get_filename_list()[0]))


# Download files
download_requests = [request.save_data(raise_download_errors=True) for request in process_requests]

# Rename filest
for file in save_locations:
    target_name = dirname(dirname(file))
    rename(file, target_name + ".tiff")
    # Remove leftover folder
    rmtree(target_name)

# # Now extract all the files (only if more than one output was defined)
# abc = [request.get_filename_list()[0] for request in process_requests]
# for image in abc:
#     tar_path = join(data_folder, image)
#     with tarfile.open(tar_path) as my_tar:
#         my_tar.extractall(data_folder)
#     rmtree(dirname(tar_path))

Wall time: 1min 22s


## Visualize results in Jupyter Notebook

TBA

## Download numpy and save GeoTIFF with rasterio

TBA